<a href="https://colab.research.google.com/github/blockchainrelativity/Colab_Experiments/blob/main/OpenAI_Assistants_API_Streaming_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install and Configure OpenAI Client

In [ ]:
%%capture
!pip install openai

In [ ]:
from openai import OpenAI
from getpass import getpass
client = OpenAI(api_key=getpass("Your OpenAI Key: "), timeout=10)

Your OpenAI Key: ··········


# Chat Completions API Example
https://cookbook.openai.com/examples/how_to_stream_completions

In [ ]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {'role': 'user', 'content': "What's 1+1? Answer in one word."}
    ],
    temperature=0,
    stream=True
)

In [ ]:
for chunk in response:
    # print(chunk)
    print(chunk.choices[0].delta.content)
    print("****************")


****************
2
****************
None
****************


# Assistants API Example
https://platform.openai.com/docs/assistants/overview?context=with-streaming

### Step 1: Create an Assistant
An Assistant represents an entity that can be configured to respond to a user's messages using several parameters like model, instructions, and tools.

In [ ]:


assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4-turbo-preview",
)

Your OpenAI Key: ··········


### Step 2: Create a Thread
A Thread represents a conversation between a user and one or many Assistants. You can create a Thread when a user (or your AI application) starts a conversation with your Assistant.

In [ ]:
thread = client.beta.threads.create()

### Step 3: Add a Message to the Thread
The contents of the messages your users or applications create are added as Message objects to the Thread. Messages can contain both text and files. There is no limit to the number of Messages you can add to Threads — we smartly truncate any context that does not fit into the model's context window.

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

### Step 4: Create a Run
Once all the user Messages have been added to the Thread, you can Run the Thread with any Assistant. Creating a Run uses the model and tools associated with the Assistant to generate a response. These responses are added to the Thread as assistant Messages.

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

# Then, we use the `create_and_stream` SDK helper
# with the `EventHandler` class to create the Run
# and stream the response.

with client.beta.threads.runs.create_and_stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

from sympy import symbols, Eq, solve

# Define the symbol
x = symbols('x')

# Define the equation
equation = Eq(3*x + 11, 14)

# Solve the equation
solution = solve(equation, x)

solution

output >

[1]

assistant > The solution to the equation \(3x + 11 = 14\) is \(x = 1\).

# Example with Custom Functions

### Step 1: Create assistant with custom tools

In [ ]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4-turbo-preview",
  tools=[{
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }, {
    "type": "function",
    "function": {
      "name": "getNickname",
      "description": "Get the nickname of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
        },
        "required": ["location"]
      }
    }
  }]
)

### Step 2: Create a thread

In [ ]:
thread = client.beta.threads.create()

### Step 3: Add message to thread

In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Check weather in New York and get nickname of this city."
)

### Step 4: Run thread

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler
import asyncio

# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.

class EventHandler(AssistantEventHandler):
  tool_outputs = []
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

  # def on_tool_call_done(self, tool_call: ToolCall) -> None:
  #   if tool_call.function.name == "getCurrentWeather":
  #     self.tool_outputs.append({"tool_call_id": tool_call.id, "output": "The weather is 75 degrees."})


run = None

with client.beta.threads.runs.create_and_stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()
  run = stream.get_final_run()


assistant > function


assistant > function



### Step 5: Get tool outputs

In [ ]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls
tool_calls

[RequiredActionFunctionToolCall(id='call_LzUxIWb0RmUjjkJ3v1QtJk4V', function=Function(arguments='{"location": "New York, NY", "unit": "f"}', name='getCurrentWeather'), type='function'),
 RequiredActionFunctionToolCall(id='call_1co91yJSr69bHAnb4NCelOof', function=Function(arguments='{"location": "New York, NY"}', name='getNickname'), type='function')]

In [ ]:
tool_outputs = []
for tool_call in tool_calls:
    if tool_call.function.name == "getCurrentWeather":
        tool_outputs.append({"tool_call_id": tool_call.id, "output": "The weather is 75 degrees"})
    if tool_call.function.name == "getNickname":
        tool_outputs.append({"tool_call_id": tool_call.id, "output": "The big apple"})

### Step 6: Submit Tool Outputs

In [ ]:
with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler()
) as stream:
    stream.until_done()


assistant > The current weather in New York, NY, is 75 degrees Fahrenheit, and the city's nickname is "The Big Apple."

# Agency Swarm Implementation.

Fore more details, see: https://vrsen.github.io/agency-swarm/advanced-usage/agencies/#streaming-responses

### Install Agency Swarm

In [ ]:
%%capture
!pip install git+https://github.com/VRSEN/agency-swarm.git gradio

In [ ]:
from agency_swarm import set_openai_client
set_openai_client(client)

### Step 1: Define your tools.
Use `BaseTool` with your logic inside

In [ ]:
from agency_swarm.tools import BaseTool
from pydantic import Field

class GetCurrentWeather(BaseTool):
    """
    Get the weather in a specified location.
    """
    location: str = Field(
        ..., description="The city and state e.g. San Francisco, CA"
    )
    unit: str = Field(
        default="f", description="The unit of temperature (c for Celsius, f for Fahrenheit)"
    )

    def run(self) -> str:
        """
        Return a fixed temperature of 75 degrees Fahrenheit.
        """
        return "75 degrees"

class GetNickname(BaseTool):
    """
    Get the nickname of a city.
    """
    location: str = Field(
        ..., description="The city and state e.g. San Francisco, CA"
    )

    def run(self) -> str:
        """
        Return "The Big Apple" as the nickname.
        """
        return "The Big Apple"


### Step 2: Create your Agent
Most of the properties are basically the same, with a few additional ones like `files_folder` to simplify the agent creation process

In [ ]:
from agency_swarm import Agency, Agent

agent = Agent(
      instructions="You are a weather bot. Use the provided functions to answer questions.",
      model="gpt-4-turbo-preview",
      tools=[GetCurrentWeather, GetNickname]
)

### Step 3: Initialize your Agency
In this example we'll just use 1 agent. (Yes, you can do that)

In [ ]:
agency = Agency([
            agent,
        ])

### Step 3: Define Event Handler
The only difference is that you must extend the AgencyEventHandler class, which has 2 additional properties: `agent_name` and `recipient_agent_name`, to get the names of the agents communicating with each other. (See the on_text_created below.)

In [ ]:
from typing_extensions import override
from agency_swarm import AgencyEventHandler

class EventHandler(AgencyEventHandler):
    @override
    def on_text_created(self, text) -> None:
        # get the name of the agent that is sending the message
        print(f"\n{self.recipient_agent_name} @ {self.agent_name}  > ", end="", flush=True)

    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\n{self.recipient_agent_name} > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
            if delta.code_interpreter.outputs:
                print(f"\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs:
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)

    @classmethod
    def on_all_streams_end(cls):
        print("\n\nAll streams have ended.") # Conversation is over and message is returned to the user.

Also, there is an additional class method `on_all_streams_end` which is called when all streams have ended. This method is needed because, unlike in the official documentation, your event handler will be called multiple times and probably by even multiple agents.

### Step 4: Get Completion

In [ ]:
response = agency.get_completion_stream("Check weather in New York and get nickname of this city.", event_handler=EventHandler)

THREAD:[ user -> Agent ]: URL https://platform.openai.com/playground?assistant=asst_rCF4gwi6UmU5IKJawNB6Ghwp&mode=assistant&thread=thread_hPbbXgQyVTBXPFZ2RMoCbJOy

Agent > function


Agent > function


Agent @ User  > The current weather in New York, NY is 75 degrees Fahrenheit, and the nickname of the city is "The Big Apple."

All streams have ended.


OR

### Simply run gradio demo

In [ ]:
agency.demo_gradio(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fbc02961f238e66eba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Gradio Blocks instance: 6 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x7d65136bb520>
 |-<gradio.components.chatbot.Chatbot object at 0x7d65136bb430>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x7d65136bb520>
 |-<gradio.components.chatbot.Chatbot object at 0x7d65136bb430>
fn_index=1
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x7d65136bb520>
 |-<gradio.components.chatbot.Chatbot object at 0x7d65136bb430>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x7d65136bb520>
 |-<gradio.components.chatbot.Chatbot object at 0x7d65136bb430>
fn_index=2
 inputs:
 |-<gradio.components.dropdown.Dropdown object at 0x7d65136bb460>
 outputs:
fn_index=3
 inputs:
 |-<gradio.templates.Files object at 0x7d65136e8e50>
 outputs:
fn_index=4
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x7d65136bb520>
 |-<gradio.components.chatbot.Chatbot object at 0x7d65136bb430>
 outputs:
 |-